In [39]:
!pip install pandas openpyxl
!pip install -U scikit-learn
!pip install matplotlib
!pip install seaborn


In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer



In [92]:
file_path = '/Users/arjunjohal/customer-churn/Telco_customer_churn.xlsx'
df = pd.read_excel(file_path)
print(df.head())

   CustomerID  Count        Country       State         City  Zip Code  \
0  3668-QPYBK      1  United States  California  Los Angeles     90003   
1  9237-HQITU      1  United States  California  Los Angeles     90005   
2  9305-CDSKC      1  United States  California  Los Angeles     90006   
3  7892-POOKP      1  United States  California  Los Angeles     90010   
4  0280-XJGEX      1  United States  California  Los Angeles     90015   

                 Lat Long   Latitude   Longitude  Gender  ...        Contract  \
0  33.964131, -118.272783  33.964131 -118.272783    Male  ...  Month-to-month   
1   34.059281, -118.30742  34.059281 -118.307420  Female  ...  Month-to-month   
2  34.048013, -118.293953  34.048013 -118.293953  Female  ...  Month-to-month   
3  34.062125, -118.315709  34.062125 -118.315709  Female  ...  Month-to-month   
4  34.039224, -118.266293  34.039224 -118.266293    Male  ...  Month-to-month   

  Paperless Billing             Payment Method  Monthly Charges Tota

In [93]:
dropped = df['Churn Label']
dropped.describe()

count     7043
unique       2
top         No
freq      5174
Name: Churn Label, dtype: object

In [94]:
df_clean = df.drop(columns=['CustomerID', 'Lat Long', 'Country', 'State', 'City', 'Count', 'Churn Label', 'Churn Score', 'CLTV', 'Churn Reason', 'Churn Value'])
df_clean.get('Tenure Months').dtype

dtype('int64')

In [99]:
# Convert numeric columns safely
for c in num_cols:
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")

In [100]:
X_train_val, X_test, y_train_val, y_test = train_test_split(df_clean, dropped, test_size = 0.2, random_state = 42, stratify=dropped)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.25, random_state = 42, stratify=y_train_val)

In [101]:
num_cols = ['Monthly Charges', 'Total Charges', 'Tenure Months']
cat_cols = df_clean.columns.difference(num_cols)


In [102]:


num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols),
    ]
)

model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

In [104]:
model.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
#y_proba = model.predict_proba(X_val)[:, 1]
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

          No       0.88      0.78      0.83      1035
         Yes       0.53      0.70      0.60       374

    accuracy                           0.76      1409
   macro avg       0.71      0.74      0.71      1409
weighted avg       0.79      0.76      0.77      1409



/Users/arjunjohal/customer-churn/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [5, 6, 18] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
